In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ReadData
from io import BytesIO
import openpyxl

In [3]:
path = 'CAOWijzer dataset 2017 tm 2021P06.xlsx'

In [4]:
writer = pd.ExcelWriter('VerdelingKlanten.xlsx', engine='openpyxl', mode='a')

In [5]:
app = ReadData.ReadData(path_file=path)

In [6]:
app.process_customers()

In [7]:
first_df = app.get_dataframe_from_customer_list()

In [8]:
first_df = first_df[first_df['Abonnement'].notna()]

In [12]:
first_df = first_df.sort_values(by='Laatst gefactureerde datum')

In [9]:
# making the df to active customers
first_df = first_df[first_df['Laatst gefactureerde datum'] == first_df['Laatst gefactureerde datum'].unique()[-1]]

In [10]:
first_df.to_excel(writer, sheet_name='Data_punt_3_huidigeVerdeling', startcol=4)

In [11]:
light_threshold = 17.14
standard_threshold = 45

In [12]:
updated_df = ReadData.update_subscriptions(data_df=first_df, light_threshold=light_threshold, standard_threshold=standard_threshold)

In [13]:
updated_df.to_excel(writer, sheet_name='Data_punt_3_naVerdelingHuidig', startcol=4)

In [14]:
subs = first_df['Abonnement'].unique()
subs

array(['Light', 'Standard', 'Premium'], dtype=object)

In [48]:
amount_sleeping = len(first_df[first_df['Gem. Gebruik'] == 0])
amount_light = len(first_df[(first_df['Abonnement'] == subs[0]) & (first_df['Gem. Gebruik'] > 0)])
amount_standard = len(first_df[(first_df['Abonnement'] == subs[1]) & (first_df['Gem. Gebruik'] > 0)])
amount_premium = len(first_df[(first_df['Abonnement'] == subs[2]) & (first_df['Gem. Gebruik'] > 0)])
values_normal = ['Huidig', amount_sleeping, amount_light, amount_standard, amount_premium]

In [16]:
updated_df.loc[(updated_df['Gem. Gebruik'] < light_threshold), 'Abonnement'] = 'Light'
updated_df.loc[(updated_df['Gem. Gebruik'] > light_threshold) & (updated_df['Gem. Gebruik'] < standard_threshold) , 'Abonnement'] = 'Standard'
updated_df.loc[(updated_df['Gem. Gebruik'] > standard_threshold), 'Abonnement'] = 'Premium'

In [22]:
amount_sleeping2 = len(updated_df[updated_df['Gem. Gebruik'] == 0])
amount_light2 = len(updated_df[updated_df['Abonnement'] == subs[0]]) - amount_sleeping
amount_standard2 = len(updated_df[updated_df['Abonnement'] == subs[1]])
amount_premium2 = len(updated_df[updated_df['Abonnement'] == subs[2]])
values_updated = ['Na verdeling', 0, amount_light2, amount_standard2, amount_premium2]

In [20]:
cols = ['Situatie', 'Slapend', 'Light', 'Standard', 'Premium']
result_df = pd.DataFrame(columns=cols)

In [23]:
result_df.loc[len(result_df)] = values_normal
result_df.loc[len(result_df)] = values_updated

In [24]:
result_df = result_df.set_index('Situatie')

In [25]:
sheet_name = 'Data_punt_3_naVerdelingHuidig'
result_df.to_excel(writer, sheet_name=sheet_name)

image_name = BytesIO()

ax = result_df.T.plot(kind='bar', rot=360)
for p in ax.patches:
    if int(p.get_height()) > 0:
        ax.annotate(str(p.get_height()), (p.get_x() + 0.05, p.get_height() + 0.75))

plt.title('Verdeling na bepaling snijpunten\n(zonder slapende klanten)')
plt.xlabel(f'\nMet snijpunten:\nLight: {light_threshold} - Standard: {standard_threshold}.\nberekend obv van de huidige meerprijzen')

plt.savefig(image_name,  bbox_inches='tight', pad_inches=0.1, format='png')
plt.close()

image_name.seek(0)

worksheet = writer.sheets[sheet_name]
image = openpyxl.drawing.image.Image(image_name)
image.anchor = 'B2'
worksheet.add_image(image)

writer.save()

In [22]:
# Data voor punt 5

In [23]:
data_df = updated_df[updated_df['Abonnement'] == 'Light']
data_df.to_excel(writer, sheet_name='Data_punt_5', startcol=4)

In [24]:
image_name = BytesIO()

from matplotlib.ticker import AutoMinorLocator
fig, ax = plt.subplots(1, figsize=(20,6))
bin_amount = 17
n, bins, patches = plt.hist(data_df['Gem. Gebruik'], bins=bin_amount, zorder=2, rwidth=0.9)

plt.title('Histogram van gemiddelde kliks van klanten in Light abonnement')
plt.xlabel('\nAantal gemiddelde kliks\n\n(Na klantverdeling obv huidige tarieven, kliks zijn afgerond naar hele getallen, slapende klanten zijn hierin meegenomen)')

# define minor ticks and draw a grid with them
minor_locator = AutoMinorLocator(2)
plt.gca().xaxis.set_minor_locator(minor_locator)
# plt.grid(which='minor', color='white', lw = 0.5)

# x ticks
xticks = [(bins[idx+1] + value)/2 for idx, value in enumerate(bins[:-1])]
xticks_labels = [ "{:.0f} tot {:.0f}".format(value, bins[idx+1]) for idx, value in enumerate(bins[:-1])]

plt.xticks(xticks, labels = xticks_labels)

# remove major and minor ticks from the x axis, but keep the labels
ax.tick_params(axis='x', which='both',length=0)

# remove y ticks
plt.yticks([])

# plot values on top of bars
for idx, value in enumerate(n):
    if value > 0:
        plt.text(xticks[idx], value+0.5, int(value), ha='center')

# set ylabel
plt.ylabel('Aantal klanten')


plt.savefig(image_name,  bbox_inches='tight', pad_inches=0.1, format='png')
plt.close()

image_name.seek(0)

worksheet = writer.sheets['Data_punt_5']
image = openpyxl.drawing.image.Image(image_name)
image.anchor = 'B2'
worksheet.add_image(image)

writer.save()
writer.close()

In [26]:
# ONDERDEEL 2

In [27]:
light_threshold = 15
standard_threshold = 37.5
column_name = 'Gem. Gebruik'

In [28]:
updated_df = ReadData.update_subscriptions(data_df=first_df, light_threshold=light_threshold, standard_threshold=standard_threshold)

In [29]:
updated_df.to_excel(writer, sheet_name='Data_punt_3_naVerdelingPDF', startcol=4)

In [36]:
amount_light2 = len(updated_df[updated_df['Abonnement'] == subs[0]])
amount_standard2 = len(updated_df[updated_df['Abonnement'] == subs[1]])
amount_premium2 = len(updated_df[updated_df['Abonnement'] == subs[2]])
values_updated = ['Na verdeling', 0, amount_light2, amount_standard2, amount_premium2]

In [49]:
cols = ['Situatie', 'Slapend', 'Light', 'Standard', 'Premium']
result_df = pd.DataFrame(columns=cols)

In [53]:
values_normal = ['Huidig', 11, 61, 132, 11]

In [55]:
values_normal = ['Huidig', 11, amount_light, amount_standard, amount_premium]
result_df.loc[len(result_df)] = values_normal
result_df.loc[len(result_df)] = values_updated

In [56]:
result_df = result_df.set_index('Situatie')

In [57]:
sheet_name = 'Data_punt_3_naVerdelingPDF'
result_df.to_excel(writer, sheet_name=sheet_name)

image_name = BytesIO()

ax = result_df.T.plot(kind='bar', rot=360)
for p in ax.patches:
    if int(p.get_height()) > 0:
        ax.annotate(str(p.get_height()), (p.get_x() + 0.05, p.get_height() + 0.75))

plt.title('Verdeling na bepaling snijpunten\n')
plt.xlabel(f'\nMet snijpunten:\nLight: {light_threshold} - Standard: {standard_threshold}.\nberekend obv de voorgestelde prijzen vanuit het pdf-bestand')

plt.savefig(image_name,  bbox_inches='tight', pad_inches=0.1, format='png')
plt.close()

image_name.seek(0)

worksheet = writer.sheets[sheet_name]
image = openpyxl.drawing.image.Image(image_name)
image.anchor = 'B2'
worksheet.add_image(image)

writer.save()

In [24]:
writer.close()